<a href="https://colab.research.google.com/github/Mansi06Salar/Coronary-Artery-Disease-Detection-using-AI/blob/main/Applying_10_ML_models_on_preprocessed_(MinMax_scaler)_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.24.3 catboost==1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.24.3 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.3 which is incompatible.
albucore 0.0.23 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which i

In [ ]:
import numpy as np
import pandas as pd
import time
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
df = pd.read_csv("Reduced_Dataset_Hybrid_Approach.csv")
print(df.head())

         TG  DLP   DM       LDL       PLT      Neut   BP     Lymph       Age  \
0  0.210267  1.0  0.0  0.640187  0.329149  0.350877  0.2  0.603774  0.410714   
1  0.268509  0.0  0.0  0.481308  0.195258  0.403509  0.5  0.584906  0.660714   
2  0.065153  0.0  0.0  0.242991  0.285914  0.491228  0.1  0.584906  0.428571   
3  0.025666  0.0  0.0  0.172897  1.000000  0.701754  0.1  0.207547  0.642857   
4  0.131293  0.0  0.0  0.429907  0.347280  0.122807  0.2  0.905660  0.357143   

     Weight  ...         K  Tinversion  Region_RWMA  Current_Smoker  HTN  \
0  0.583333  ...  0.472222         1.0          0.0             1.0  1.0   
1  0.305556  ...  0.472222         1.0          1.0             0.0  1.0   
2  0.083333  ...  0.472222         0.0          0.5             1.0  0.0   
3  0.263889  ...  0.388889         0.0          0.0             0.0  1.0   
4  0.541667  ...  0.277778         0.0          0.0             0.0  1.0   

   Atypical  St_Depression       FBS       ESR  CAD  
0       

In [ ]:
#Fix LightGBM warning: Remove spaces from feature names
df.columns = [col.replace(" ", "_") for col in df.columns]

In [ ]:
X = df.drop(columns=['CAD'])
y = df['CAD']

In [ ]:
print("Class Distribution BEFORE SMOTE:", Counter(y))

Class Distribution BEFORE SMOTE: Counter({1: 216, 0: 87})


In [ ]:
smote_applied = False
imbalance_threshold = 0.6  #SMOTE if any class is <40% of the largest class

class_counts = Counter(y)
minority_class = min(class_counts, key=class_counts.get)
majority_class = max(class_counts, key=class_counts.get)
imbalance_ratio = class_counts[minority_class] / class_counts[majority_class]

if imbalance_ratio < imbalance_threshold:
    smote = SMOTE(random_state=42)
    X, y = smote.fit_resample(X, y)
    smote_applied = True

print("Class Distribution AFTER SMOTE:", Counter(y) if smote_applied else "SMOTE Not Applied")

Class Distribution AFTER SMOTE: Counter({1: 216, 0: 216})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(max_depth=10),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=10),
    "XGBoost": XGBClassifier(eval_metric='logloss', use_label_encoder=False),
    "LightGBM": LGBMClassifier(max_depth=10, min_data_in_leaf=5),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(n_estimators=50),
    "SVM": SVC(probability=True),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naïve Bayes": GaussianNB()
}

In [ ]:
results = []
for name, model in models.items():
    try:
        start_time = time.time()
        model.fit(X_train, y_train)
        train_time = time.time() - start_time

        y_pred = model.predict(X_test)

        y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average="binary", zero_division=1)
        recall = recall_score(y_test, y_pred, average="binary")
        f1 = f1_score(y_test, y_pred, average="binary")
        auc = roc_auc_score(y_test, y_prob) if y_prob is not None else "N/A"

        results.append([name, accuracy, precision, recall, f1, auc, train_time])

    except Exception as e:
        print(f"Error training {name}: {e}")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:58:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Info] Number of positive: 173, number of negative: 172
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000709 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 984
[LightGBM] [Info] Number of data points in the train set: 345, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501449 -> initscore=0.005797
[LightGBM] [Info] Start training from score 0.005797
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] min_data_in_leaf is set=5, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=5
[LightGBM] [Warn

In [ ]:
metrics_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1 Score", "AUROC", "Train Time"])
metrics_df = metrics_df.sort_values(by="Accuracy", ascending=False)

print("\nModel Performance Summary:")
print(metrics_df.to_string(index=False))



Model Performance Summary:
              Model  Accuracy  Precision   Recall  F1 Score    AUROC  Train Time
           AdaBoost  0.942529   0.975000 0.906977  0.939759 0.966173    0.153568
            XGBoost  0.919540   0.950000 0.883721  0.915663 0.975687    1.420854
      Random Forest  0.908046   0.926829 0.883721  0.904762 0.972516    1.105048
           CatBoost  0.896552   0.925000 0.860465  0.891566 0.968288    8.892792
        Naïve Bayes  0.896552   0.925000 0.860465  0.891566 0.933932    0.002794
Logistic Regression  0.885057   0.883721 0.883721  0.883721 0.932347    0.033205
           LightGBM  0.885057   0.945946 0.813953  0.875000 0.972516    0.318028
                SVM  0.862069   0.860465 0.860465  0.860465 0.932347    0.022469
      Decision Tree  0.827586   0.850000 0.790698  0.819277 0.827167    0.042372
                KNN  0.827586   0.868421 0.767442  0.814815 0.894820    0.002293


In [ ]:
import psutil

ram_info = psutil.virtual_memory()
total_ram = ram_info.total / (1024 ** 3)

cpu_count = psutil.cpu_count(logical=True)
cpu_freq = psutil.cpu_freq().max

print(f"CPU Cores: {cpu_count}")
print(f"Max CPU Frequency: {cpu_freq:.2f} MHz")
print(f"Total RAM: {total_ram:.2f} GB")


💻 CPU Cores: 2
⚡ Max CPU Frequency: 0.00 MHz
🖥️ Total RAM: 12.67 GB


In [ ]:
!lscpu


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt
                           aes xsave avx f16c rdrand hypervisor 